In [1]:
import os
import h5py
import numpy as np
from multiprocessing import Pool


In [2]:
PATH = '/home/lorenzo/phd/NS_HOT_EOS/EOS/compOSE/FOP(SFHoY)'
h5_name = 'TEST_' + PATH.split('/')[-2] + '.h5'
OUTPUT = os.path.join(PATH, h5_name)
# files = [file for file in os.listdir(PATH) if 'eos.' in file and not '.pdf' in file and not '.init' in file]
# files

files = ['eos.nb', 'eos.thermo', 'eos.t', 'eos.yq']

skip_rows = {
                'eos.nb' : 2,
                'eos.thermo' : 1,
                'eos.t' : 2,
                'eos.yq' : 2
            }

In [3]:
data = {}
for file in files:
    FILE_PATH = os.path.join(PATH, file)
    data[file] = np.loadtxt(FILE_PATH, skiprows=skip_rows[file])
    if file == 'eos.thermo':
        with open(FILE_PATH) as f:
            m_n, m_p, _ = np.fromstring(f.readline().strip('\n'), dtype = float, sep='\t')

In [4]:
# Gli indici sono da 1 a N in Fortran, da 0 a N-1 in Python
index_T    = data['eos.thermo'][:, 0].astype(int) - 1
index_nb   = data['eos.thermo'][:, 1].astype(int) - 1
index_ye   = data['eos.thermo'][:, 2].astype(int) - 1

logrho     = np.log10(data['eos.nb'] * m_n)  # log_10( nb * m_n ), nb exponential   MeV / fm^3
pointsrho  = len(logrho)

y_e        = data['eos.yq']                  # LINEARE                 Adimensionale
pointsye   = len(y_e)

logtemp    = np.log10(data['eos.t'])         # log_10( T ), T exponential           MeV
pointstemp = len(logtemp)

In [5]:
maps = lambda i,j,k : i + pointsrho * pointsye * j + pointsye * k


## COMPUTE PRESSURE, RESHAPE AND TEST FOR EQUIVALENCE!!! Secondo metodo circa 28 volte più veloce del primo

pressure   = data['eos.thermo'][:, 3] * data['eos.nb'][index_nb]   # MeV / fm^3
pp = np.zeros((pointsye, pointstemp, pointsrho))
for i in range(pointsye):
    for j in range(pointstemp):
        for k in range(pointsrho):
            pp[i, j, k] = pressure[maps(i,j,k)]

pressure_r = pressure[maps(np.arange(pointsye)[:, None, None], np.arange(pointstemp)[None, :, None], np.arange(pointsrho)[None, None, :]) ]

##### NEW
pressure_shift = np.abs(np.min([np.min(pressure_r), 0])) * 1.01
pressure_s = pressure_r + pressure_shift
#####

np.testing.assert_allclose(pressure_r, pp)

#############################################
## COMPUTE ENTROPY, RESHAPE AND TEST FOR EQUIVALENCE!!! Secondo metodo circa 28 volte più veloce del primo

entropy    = data['eos.thermo'][:, 4] * data['eos.nb'][index_nb]   # Adimensionale
ee = np.zeros((pointsye, pointstemp, pointsrho))
for i in range(pointsye):
    for j in range(pointstemp):
        for k in range(pointsrho):
            ee[i, j, k] = entropy[maps(i,j,k)]

entropy_r = entropy[maps(np.arange(pointsye)[:, None, None], np.arange(pointstemp)[None, :, None], np.arange(pointsrho)[None, None, :]) ]

np.testing.assert_allclose(entropy_r, ee)

#############################################
## COMPUTE ENERGY, RESHAPE AND TEST FOR EQUIVALENCE!!! Secondo metodo circa 28 volte più veloce del primo
# Se ho capito bene Q7 --> data['eos.thermo'][:, 9] è esattamente "energy" negli h5 (da capire come la scala)
energy     = data['eos.thermo'][:, 9]
en = np.zeros((pointsye, pointstemp, pointsrho))
for i in range(pointsye):
    for j in range(pointstemp):
        for k in range(pointsrho):
            en[i, j, k] = energy[maps(i,j,k)]

energy_r = energy[maps(np.arange(pointsye)[:, None, None], np.arange(pointstemp)[None, :, None], np.arange(pointsrho)[None, None, :]) ]

energy_shift = np.abs(np.min([np.min(energy_r), 0])) * 1.01
energy_s = energy_r + energy_shift


np.testing.assert_allclose(energy_r, en)

In [16]:
e = m_n * data['eos.nb'][index_nb] * ( 1 + data['eos.thermo'][:, 9] + energy_shift )
e_r = e[maps(np.arange(pointsye)[:, None, None], np.arange(pointstemp)[None, :, None], np.arange(pointsrho)[None, None, :]) ]

gamma = np.zeros_like(e_r)
for i in range(pointsye):
    for j in range(pointstemp):
        x = np.log(e_r[i, j, :])
        y = np.log(pressure_s[i, j, :])
        gamma[i, j, :-1] = (y[1:] - y[:-1]) / (x[1:] - x[:-1])


cs2 = np.zeros_like(e_r)
for i in range(pointsye):
    for j in range(pointstemp):
        x = e_r[i, j, :]
        y = pressure_s[i, j, :]
        cs2[i, j, :-1] = (y[1:] - y[:-1]) / (x[1:] - x[:-1])

In [17]:
logenergy = np.log10(energy_s)
logpress = np.log10(pressure_s)


# cs2 = pressure_s * gamma / e_r    # Adim

In [18]:
energy_shift

0.00972144291

In [21]:
pressure_shift

0.0

In [19]:
ds = {  
        'logrho'   : logrho   , 'pointsrho'     : pointsrho, 
        'y_e'      : y_e      , 'pointsye'      : pointsye, 
        'logtemp'  : logtemp  , 'pointstemp'    : pointstemp, 
        'logpress' : logpress , 'pressure_shift': pressure_shift,
        'entropy'  : entropy_r, 
        'logenergy': logenergy, 'energy_shift'  : energy_shift,
        'gamma'    : gamma    , 'cs2'           : cs2
    }

In [20]:
import os
os.system(f'rm {OUTPUT}')

with h5py.File(OUTPUT, "w") as f:
    for d in ds:
        dset = f.create_dataset(d, data=ds[d], dtype='f')

sh: 1: Syntax error: "(" unexpected
